<a href="https://colab.research.google.com/github/AmbiTyga/73String/blob/main/73String.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scrapy -q

     |████████████████████████████████| 245kB 10.6MB/s 
     |████████████████████████████████| 61kB 3.9MB/s 
     |████████████████████████████████| 3.2MB 38.8MB/s 
     |████████████████████████████████| 3.2MB 26.5MB/s 
     |████████████████████████████████| 3.1MB 49.1MB/s 
     |████████████████████████████████| 245kB 37.1MB/s 
     |████████████████████████████████| 61kB 4.5MB/s 
     |████████████████████████████████| 81kB 5.2MB/s 


In [2]:
import pandas as pd
import numpy as np
import os
import glob
import matplotlib.pyplot as plt

In [7]:
data = pd.read_excel('/content/Training_Data.01 (1).xlsx')

In [8]:
data.head()

,Company Name,Business Description,Industry Classification Tag
0,"ADSOUTH PARTNERS, INC.","Adsouth Partners, Inc. provides advertising ag...",Advertising
1,"Artec Global Media, Inc.","Artec Global Media, Inc., formerly Artec Consu...",Advertising
2,Betawave Corp.,Betawave Corporation provides online marketing...,Advertising
3,BOSTON OMAHA Corp,Boston Omaha Corporation is engaged in the bus...,Advertising
4,Bright Mountain Media Inc,"Bright Mountain Media, Inc. is a digital media...",Advertising


In [7]:
data['Industry Classification Tag'].unique()

array(['Advertising', 'Aerospace & Defense', 'Apparel Retail',
       'Apparel, Accessories & Luxury Goods', 'Application Software',
       'Asset Management & Custody Banks', 'Auto Parts & Equipment',
       'Biotechnology', 'Building Products', 'Casinos & Gaming',
       'Commodity Chemicals', 'Communications Equipment',
       'Construction & Engineering',
       'Construction Machinery & Heavy Trucks', 'Consumer Finance',
       'Data Processing & Outsourced Services',
       'Diversified Metals & Mining', 'Diversified Support Services',
       'Electric Utilities', 'Electrical Components & Equipment',
       'Electronic Equipment & Instruments',
       'Environmental & Facilities Services', 'Gold',
       'Health Care Equipment', 'Health Care Facilities',
       'Health Care Services', 'Health Care Supplies',
       'Health Care Technology', 'Homebuilding',
       'Hotels, Resorts & Cruise Lines',
       'Human Resource & Employment Services', 'Industrial Machinery',
       'Integ

In [ ]:
from w3lib.html import remove_tags
import requests
from scrapy.selector import Selector

def inc_info(keywords):
  '''
  The function googles the keywords, and returns the details from Knowledge Graph panel of Google. 
  '''
  keywords = keywords.replace(" ",'+')
  url = f'https://www.google.com/search?&q={keywords}'

  req = requests.get(url,
                   headers= {'User-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'})
  details_with_tags = Selector(text = req.text).css('span.hgKElc').get()

  if not details_with_tags:
    details_with_tags = Selector(text = req.text).css('div.kno-rdesc span').get()

  if details_with_tags:
    details = remove_tags(details_with_tags)
    return details

In [24]:
data['Business Description'].fillna(data[data['Business Description'].isna()]['Company Name'].apply(inc_info),inplace = True)

In [25]:
data[data['Business Description'].isna()]['Business Description']

1254    None
2247    None
2288    None
2327    None
2384    None
2423    None
2428    None
2434    None
3512    None
Name: Business Description, dtype: object